In [2]:
from utils import *
from detectors import torch_script_module
from carla_control import carla_client
import numpy as np
from mayavi import mlab
import torch
import time

logger = create_logger()
CFG_FILE = ".\\configs\\kitti_test_mono_file.yaml"
cfg = cfg_from_yaml_file(CFG_FILE, cfg)
print(cfg)

pcs_dataset = file_point_cloud_dataset(dataset_cfg = cfg.DATA_CONFIG, logger=logger, root_path=".np_cache/000001.bin", class_names=cfg.CLASS_NAMES)
for i in pcs_dataset.data_processor_queue:
    print(i)
    for s in pcs_dataset.data_processor_queue[i]:
        print(s)

{'ROOT_DIR': WindowsPath('D:/Python_project/beamng_python_experiment'), 'LOCAL_RANK': 0, 'CLASS_NAMES': ['Car', 'Pedestrian', 'Cyclist'], 'DATA_CONFIG': {'POINT_CLOUD_RANGE': [0, -39.68, -3, 69.12, 39.68, 1], 'POINT_FEATURE_ENCODING': {'encoding_type': 'absolute_coordinates_encoding', 'used_feature_list': ['x', 'y', 'z', 'intensity'], 'src_feature_list': ['x', 'y', 'z', 'intensity']}, 'PREVIEW_CHANNEL': ['ori', 'adv'], 'DATA_PROCESSOR': {'ori': [{'NAME': 'raw_data_remain'}], 'adv': [{'NAME': 'raw_data_remain'}, {'NAME': 'utils.mods.pre_pocessing.robo3d.cross_sensor_process', 'EXT_MODULE': True, 'NUM_BEAM_TO_DROP': 32}]}}, 'MODEL': {'NAME': 'PointPillar', 'POST_PROCESSING': {'RECALL_THRESH_LIST': [0.3, 0.5, 0.7], 'SCORE_THRESH': 0.1, 'OUTPUT_RAW_SCORE': False, 'EVAL_METRIC': 'kitti', 'NMS_CONFIG': {'MULTI_CLASSES_NMS': False, 'NMS_TYPE': 'nms_gpu', 'NMS_THRESH': 0.01, 'NMS_PRE_MAXSIZE': 4096, 'NMS_POST_MAXSIZE': 500}}}}
ori
functools.partial(<bound method point_cloud_dataset_base.raw_da

In [3]:
import open3d as o3d
import threading
from utils import *

def fun(vis:o3d.cpu.pybind.visualization.Visualizer):
    print("run")
    vis.run()
    # vis.destroy_window()
    
with torch.no_grad():
    data_dict = pcs_dataset[0]
    # print(data_dict)
    for channel_name, channel in data_dict.items():
        data = data_dict[channel_name]
        # print(data_dict)
        vis = o3d.visualization.Visualizer()
        vis.create_window(window_name=channel_name)
        points = data["points"]
        draw_scenes(vis,
                    points=points, ref_boxes=None,
                    ref_scores=None, ref_labels=None, confidence=None
                )
        
        thread = threading.Thread(target=lambda: fun(vis))
        thread.start()
        

    # ori_pc = data_dict["raw_points"]
    # tran_pc = data_dict["points"]
    # print(ori_pc - tran_pc)
    # print(data_dict["points"].shape)
    # print(data_dict["intensity"].sum())
    # mlab.points3d(ori_pc[:, 0], ori_pc[:, 1], ori_pc[:, 2], data_dict["intensity"], mode='point', scale_factor=5)
    # mlab.points3d(tran_pc[:, 0], tran_pc[:, 1], tran_pc[:, 2], tran_pc[:, 3], mode='point', scale_factor=5)

    # # mlab.points3d(kitti_01_pcs[:, 0], kitti_01_pcs[:, 1], kitti_01_pcs[:, 2], kitti_01_pcs[:, 3], mode='point', scale_factor=5)
    # mlab.show(stop=False)
    # data_dict = pcs_dataset.collate_batch([data_dict])
    # print(data_dict["points"].shape)
    # load_data_to_gpu(data_dict)
    # print(data_dict["points"].size())